# Cálculo de trayectoria para nave exploratoria lunar

### Método para resolver la ODE del cohete

Para encontrar una solución numérica al problema, usamos el algoritmo de Runge-Kutta de orden 4 (RK4).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

In [ ]:
def RK4 (f, r0, t):
    """
    Implementación del algoritmo de Runge-Kutta de orden 4 para resolver sistemas
    de ecuaciones diferenciales.

    Args:
    f: función de la ED y'=f(t,y)
    r0: condición inicial
    t: discretización uniformemente espaciada del intervalo temporal.
    """
    h = t[1]-t[0]
    y = r0
    solution = np.array(r0)

    for i in range(1,len(t)):
        k1 = f(t[i-1],y)
        k2 = f(t[i-1]+0.5*h, y + 0.5*h*k1)
        k3 = f(t[i-1]+0.5*h, y + 0.5*h*k2)
        k4 = f(t[i-1]+h, y + h*k3)

        y += h*(k1+2*k2+2*k3+k4)/6.
        solution = np.vstack([solution, y])

    return solution

### Resolución del problema

Primero, se definen las constantes del problema.

In [ ]:
G = 6.67e-11    # Constante gravitacional
MT = 5.9736e+24 # Masa de la Tierra
RT = 6.3781e+6  # Radio terrestre
ML = 7.349e+22  # Masa de la Luna
RL = 1.7374e+6  # Radio de la Lunar
d = 3.844e+8    # Distancia Tierra-Luna
ome = 2.6616e-6 # Frecuencia angular de la órbita lunar

La implementación del algoritmo RK4 que hemos desarrollado permite resolver sistemas de ODEs de la forma $$\mathbf{x}' = f(t,\mathbf{x}).$$ Solo es necesario definir la función del lado derecho.

In [ ]:
def f(t,x):
    """
    t: tiempo
    x: vector de cuatro componentes.
    --- x[0]: r/d
    --- x[1]: phi
    --- x[2]: momento conjugado a x[0]
    --- x[3]: momento conjugado a x[1]
    """
    cs = np.cos(x[1]-ome*t)
    r_ = np.sqrt(1 + x[0]**2 + 2*x[0]*cs)
    Delta = G*MT/(d**3)
    mu = ML/MT
    d0 = x[2]
    d1 = x[3]/ (x[0]**2)
    d2 = (x[4]**2) / (x[0])**3 - Delta*(x[0]**(-2) + mu/(r_**3)*(x[0]-cs))
    d3 = -Delta*mu*x[0] / (r_**3) *np.sin(x[1]-ome*t)
    
    return np.array([d0, d1, d2, d3])